In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
import urllib.request
import json

### Find Closest Indexes

In [6]:
def fci(array, val, n = 1):
    na = array - val
    ind = np.argsort(abs(na))[:n*2]
    return(np.sort(ind))

### Topographic Altitude

Using only the 16 closest surrounding points

In [74]:
def talt(lat, lon):
    df = pd.read_csv('ITUFiles/P.836-6/TALT/TOPO_0DOT5.csv',header=None)
    lon = lon + 360 if lon < 0 else lon
    latl = np.arange(90.5, -91, -0.5)
    lonl = np.arange(-0.5, 361, 0.5)
    lat1, lat2, lat3, lat4 = fci(latl, lat,2)
    lon1, lon2, lon3, lon4 = fci(lonl, lon,2)
    latfi = latl[fci(latl, lat,2)]
    lonfi = lonl[fci(lonl, lon,2)]
    gridfi = df.loc[lat1:lat4, lon1:lon4].to_numpy()
    f = interp2d(lonfi, latfi, gridfi, kind='cubic')
    ta = f(lon,lat)[0]
    return ta
talt(-16.51,-68.17)

2.761580036517823

Using the whole array

In [35]:
def talt(lat, lon):
    lon = lon + 360 if lon < 0 else lon
    latl = np.arange(90.5, -91, -0.5)
    lonl = np.arange(-0.5, 361, 0.5)
    tall = pd.read_csv('ITUFiles/P.836-6/TALT/TOPO_0DOT5.csv', header=None).to_numpy()
    f = interp2d(lonl, latl, tall, kind='cubic')
    ta = f(lon,lat)[0]
    return ta
talt(-16.51,-68.17)

2.7232524980064294

Using the Bing Maps API

In [57]:
def get_elev(lat, lon):
    key = 'Ai9NLeGblweg0P_Br_bKnbLVkkRgbG6RlH4k5FrbBctSqD32KHMuFQmRyX-apadI'
    url = 'https://dev.virtualearth.net/REST/v1/Elevation/List?key=' + key + '&points=' + str(lat) + ',' + str(lon)
    response = urllib.request.urlopen(url)
    data = json.loads(response.read().decode())
    elevation = data['resourceSets'][0]['resources'][0]['elevations'][0]
    return elevation / 1000
get_elevation(-16.51,-68.17)

4.079

### Data loading

In [52]:
prob = ['01', '02', '03', '05', '1', '2', '3', '5', '10', '20', '30', '50', '60', '70', '80', '90', '95', '99']
vfiles = ['ITUFiles/P.836-6/V/V_'+x+'_v4.csv' for x in prob]
vschfiles = ['ITUFiles/P.836-6/VSCH/VSCH_'+x+'_v4.csv' for x in prob]
v = [pd.read_csv(x, header=None) for x in vfiles]
vsch = [pd.read_csv(x, header=None) for x in vschfiles]

In [76]:
def itu836(lat, lon):
    lon = lon + 360 if lon < 0 else lon
    alt = get_elev(lat, lon)
    latl = np.arange(90,-90-1.125,-1.125)
    lonl = np.arange(0,360+1.125,1.125)
    lat1, lat2 = fci(latl, lat)
    lon1, lon2 = fci(lonl, lon)
    lat1v = latl[lat1]
    lat2v = latl[lat2]
    lon1v = lonl[lon1]
    lon2v = lonl[lon2]
    iwv = []
    for i in range(len(v)):
        alti = np.array([talt(lat1v, lon1v), talt(lat1v, lon2v), talt(lat2v, lon1v), talt(lat2v, lon2v)])
        vpi = np.array([v[i].loc[lat1, lon1], v[i].loc[lat1, lon2], v[i].loc[lat2, lon1], v[i].loc[lat2, lon2]])
        vschi = np.array([vsch[i].loc[lat1, lon1], vsch[i].loc[lat1, lon2], vsch[i].loc[lat2, lon1], vsch[i].loc[lat2, lon2]])
        vi = vpi * np.exp(-(alt - alti)/(vschi))
        f = interp2d([lon1v, lon2v, lon1v, lon2v], [lat1v, lat1v, lat2v, lat2v], vi)
        iwv.append(f(lon, lat)[0])
    return iwv

In [83]:
wat_file = pd.read_csv('ITUFiles/wat676.csv')
oxy_file = pd.read_csv('ITUFiles/oxy676.csv')
fw = wat_file['fw'].to_numpy()
b1 = wat_file['b1'].to_numpy()
b2 = wat_file['b2'].to_numpy()
b3 = wat_file['b3'].to_numpy()
b4 = wat_file['b4'].to_numpy()
b5 = wat_file['b5'].to_numpy()
b6 = wat_file['b6'].to_numpy()
fo = oxy_file['fo'].to_numpy()
a1 = oxy_file['a1'].to_numpy()
a2 = oxy_file['a2'].to_numpy()
a3 = oxy_file['a3'].to_numpy()
a4 = oxy_file['a4'].to_numpy()
a5 = oxy_file['a5'].to_numpy()
a6 = oxy_file['a6'].to_numpy()

In [81]:
def itu676(f, p, rho, T):
    # p: dry air presure (hPa)
    # e: water vaour partial pressure (hPa)
    # theta (Θ): 300/T
    # T: Temperature
    theta = 300/T
    e = (rho*T)/216.7
    So = a1 * (1e-7) * p * (theta**3) * np.exp(a2 * (1 - theta))
    Sw = b1 * (1e-1) * e * (theta**3.5) * np.exp(b2 * (1 - theta))
    deltao = (a5 + a6 * theta) * (1e-4) * (p + e) * (theta**0.8)
    deltaw = 0
    deltafo_wd = a3 * (1e-4) * (p * (theta**(0.8 - a4)) + 1.1 * e * theta)
    deltafw_wd = b3 * (1e-4) * (p * (theta**b4) + b5 * e * (theta**b6))
    deltafo = np.sqrt(deltafo_wd**2 + 2.25e-6)
    deltafw = 0.535 * deltafw_wd + np.sqrt((0.217 * deltafw_wd**2) + (2.1316e-12 * (fw**2))/(theta))
    Fo = ((f)/(fo)) * (((deltafo - deltao * (fo - f))/(deltafo**2 + (fo - f)**2))+((deltafo - deltao * (fo + f))/(deltafo**2 + (fo + f)**2)))
    Fw = ((f)/(fw)) * (((deltafw - deltaw * (fw - f))/(deltafw**2 + (fw - f)**2))+((deltafw - deltaw * (fw + f))/(deltafw**2 + (fw + f)**2)))
    d = (5.6e-4) * (p + e) * (theta**0.8)
    N2Df = f * p * theta**2 * (((6.14e-5)/(d * (1+(f/d)**2)))+((1.4e-12 * p * theta**1.5)/(1 + 1.9e-5 * f**1.5)))
    N2f = sum(Fo * So) + sum(Fw * Sw) + N2Df
    y = 0.182 * f * N2f
    ywv = 0.182 * f * sum(Fw * Sw)
    yox = 0.182 * f * (sum(Fo * So) + N2Df)
    return y, ywv, yox

In [78]:
def calc_at(daily_tcwv, freq, lat, lon):
    freq = [freq] if type(freq)!=list else freq
    daily_at = [np.empty(0)] * len(freq)
    f_ref = 20.6
    p_ref = 845
    hs = get_elev(lat,lon)
    c = 0
    for f in freq:
        a = 0.2048 * np.exp(-((f-22.43)/(3.097))**2) + 0.2326 * np.exp(-((f-183.5)/(4.096))**2) + 0.2073 * np.exp(-((f-325)/(3.651))**2) - 0.1113
        b = 8.741e4 * np.exp(-0.587 * f) + 312.2 * f ** -2.38 + 0.723
        if hs < 0:
            h = 0
        elif hs >= 0 and hs < 4:
            h = hs
        else:
            h = 4
        # print(a)
        # print(b)
        for iwv in daily_tcwv:
            rho_vref = iwv / 2.38
            t_ref = 14 * np.log(0.22*iwv/2.38) + 3 + 273.15
            if f <= 20:
                a_w = (0.0176 * iwv * itu676(f, p_ref, rho_vref, t_ref)[1])/(itu676(f_ref, p_ref, rho_vref, t_ref)[1])
            else:
                a_w = (a * (h ** b) + 1)*(0.0176 * iwv * itu676(f, p_ref, rho_vref, t_ref)[1])/(itu676(f_ref, p_ref, rho_vref, t_ref)[1])
            daily_at[c] = np.append(daily_at[c], a_w)
        c += 1
    return daily_at

In [88]:
prob_nom = [0.1, 0.2, 0.3, 0.5, 1, 2, 3, 5, 10, 20, 30, 50, 60, 70, 80, 90, 95, 99]
dfindex = ['Probabilities', 'IWV', 'A20', 'A40', 'A50', 'A75']

### El Alto

In [90]:
lat = -16.51
lon = -68.17
iwv = itu836(lat, lon)
at_20, at_40, at_50, at_75 = calc_at(iwv, [20, 40, 50, 75], lat, lon)
results = pd.DataFrame(np.column_stack([prob_nom, iwv, at_20, at_40, at_50, at_75]), columns=dfindex)
results.to_csv('TCWV/ElAlto/CCDF_ITU.csv', index=False)

### La Guardia

In [91]:
lat = -17.9074
lon = -63.3266
iwv = itu836(lat, lon)
at_20, at_40, at_50, at_75 = calc_at(iwv, [20, 40, 50, 75], lat, lon)
results = pd.DataFrame(np.column_stack([prob_nom, iwv, at_20, at_40, at_50, at_75]), columns=dfindex)
results.to_csv('TCWV/LaGuardia/CCDF_ITU.csv', index=False)

### San Ignacio de Moxos

In [92]:
lat = -14.996641
lon = -65.640895
iwv = itu836(lat, lon)
at_20, at_40, at_50, at_75 = calc_at(iwv, [20, 40, 50, 75], lat, lon)
results = pd.DataFrame(np.column_stack([prob_nom, iwv, at_20, at_40, at_50, at_75]), columns=dfindex)
results.to_csv('TCWV/SIMoxos/CCDF_ITU.csv', index=False)

### UPB

In [93]:
lat = -17.399253
lon = -66.218466
iwv = itu836(lat, lon)
at_20, at_40, at_50, at_75 = calc_at(iwv, [20, 40, 50, 75], lat, lon)
results = pd.DataFrame(np.column_stack([prob_nom, iwv, at_20, at_40, at_50, at_75]), columns=dfindex)
results.to_csv('TCWV/UPB/CCDF_ITU.csv', index=False)